In [ ]:
import sys
# Assuming we are in the notebook directory add this so that we can import the library
sys.path.append('..')

import numpy as np
from elfi import *
from elfi.smc_abc import SMC
from functools import partial
import scipy.stats as ss

import matplotlib
import matplotlib.pyplot as plt

matplotlib.style.use('ggplot')
%matplotlib inline

In [ ]:
def mean(y):
    mu = np.mean(y, axis=1, keepdims=True)
    return mu

def distance(x, y):
    d = np.linalg.norm( np.array(x) - np.array(y), ord=2, axis=0)
    return d

In [ ]:
n = 100
mu = 1.6

# Set up observed data y
y = mu + np.random.randn(n)

# Plot
plt.hist(y, bins=20);

In [ ]:
# Specify the graphical model
mu = Prior('mu', ss.uniform, -3, 10)
Y = Model('Y', 'normal', mu, observed=y)
S1 = Summary('S1', mean, Y)
d = Discrepancy('d', distance, S1)

# Specify the number of simulations and set up rejection sampling
N = 1000
smc = SMC(N, d, [mu], 10000)
n_populations = 4

In [ ]:
%time result = smc.infer(n_populations, [0.1, 0.1, 0.1, 0.1])
mu_post = result['samples'][0]

print("Samples: {} ({:.2f}%)".format(len(mu_post), len(mu_post)/N*100))

In [ ]:
fig, ax = plt.subplots(ncols=n_populations, sharex=True, sharey=True, figsize=(16,6))
for ii in range(n_populations-1):
    ax[ii].hist(result['samples_history'][ii], bins=20)
    ax[ii].set_title("Population {}".format(ii+1))
ax[n_populations-1].hist(mu_post, bins=20)
ax[n_populations-1].set_title("Population {}".format(n_populations))